# Generative Text Explainer with HuggingFace RoBERTa
This notebook demonstrates the use of prompt explainers for the HuggingFace RoBERTa model.

In [ ]:
from interpret_text.generative.lime_tools.explainers import LocalExplanationLikelihood
from interpret_text.generative.model_lib.hf_tooling import HF_LM
import transformers
import torch

# Introduction
This notebook illustrates the application of interpret-text for local analysis with HuggingFace RoBERTa, specifically in scenarios where we have access to log probabilities. It guides through essential API interactions to extract importance scores for various segments of a prompt.

Note:
- The 'LocalExplanationLikelihood' is particularly used for models that return log probabilities

Here we provide a prompt that that we need to be intrepreted

In [ ]:
# Dummy text for testing
dummy_text = """Answer the question given the context.
context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?"""

# Load Model

Create a HuggingFace RoBERTa model wrapper.

In [ ]:
def load_model(model_name: str):
    print("Loading model...")
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
    model = transformers.AutoModelForCausalLM.from_pretrained(
        model_name, trust_remote_code=True, torch_dtype=torch.bfloat16)
    
    model = model.eval()
    model = model.to("cuda")
    model_wrapped = HF_LM(model, tokenizer, device="cuda")
    return model_wrapped

model_wrapped = load_model("roberta-base")

Get the wrapped model's output.

In [ ]:
prompt = dummy_text
max_completion = 50  # Define max tokens for the completion
completion = model_wrapped.sample([prompt], max_new_tokens=max_completion)[0]

# Local Importances

Local importances are the most and least important words for a single document.

In [ ]:
# Initialize the explainer
explainer = LocalExplanationLikelihood(perturbation_model="removal",
                                       partition_fn="sentences",
                                       partition_kwargs={},
                                       progress_bar=None)

# Perform the explanation
attribution, parts = explainer.attribution(
    model_wrapped,
    prompt,
    completion,
)

In [ ]:
# Print the explanation results
print("attribution_scores contain (feat_idx, score) pairs: \n", attribution)
print("\nlist of parts from the prompt: \n", parts)